In [1]:
import pandas as pd
import numpy as np
from itertools import chain
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [2]:
from sensor_analysis_functions import fault_detection, plotting_anem_parameter_tunning, anemometer_identification, plot_sensors, get_index_list, split_kneighbor_indexes, generate_dashboard

In [3]:
df_D161075_2016 = pd.read_csv('../Cleaner/data/D161075_2016_data.csv', index_col=0)

In [4]:
df_D161075_2016.sort_values('Timestamp', inplace=True)
df_D161075_2016.reset_index(drop=True, inplace=True)

In [5]:
df_D161075_redundant_sensors = {
    'Ch1Anem': 'Ch2Anem',
    'Ch2Anem': 'Ch1Anem',
    'Ch3Anem': 'Ch4Anem',
    'Ch4Anem': 'Ch3Anem',
    'Ch5Anem': 'Ch6Anem',
    'Ch6Anem': 'Ch5Anem',
}

In [6]:
df_D161075_2016.head()

,Ch1Anem,Ch2Anem,Ch3Anem,Ch4Anem,Ch5Anem,Ch6Anem,Ch7Vane,Ch8Vane,Ch9Vane,Timestamp
0,1.2821,1.3167,1.1684,1.2491,0.9931,1.1058,235.7346,226.7614,194.9809,2016-09-14 13:40:00
1,2.1654,2.1103,2.3427,2.3701,2.4488,2.5184,285.9317,286.5346,285.9206,2016-09-14 13:50:00
2,2.7435,2.8815,2.7822,2.9845,2.7012,2.9007,254.3770,254.0049,254.8364,2016-09-14 14:00:00
3,2.6704,2.8993,2.6027,2.8005,2.5075,2.6744,242.9378,247.5312,247.7033,2016-09-14 14:10:00
4,3.0413,2.9716,3.2105,3.1648,3.3970,3.2587,127.0839,126.2498,119.0629,2016-09-14 14:20:00


In [9]:
# Slice de las primeras 7 columnas para no agarrar las columnas con los min y std
df_D161075_2016_fault_detection = fault_detection(df_D161075_2016, sensor='anem', correlation_window=3, ratio_th = 10, correlation_th= .5)


100%|██████████| 15/15 [01:52<00:00,  7.00s/it]

In [11]:
df_D161075_2016_anemometer_identification = anemometer_identification(df_D161075_2016_fault_detection, df_D161075_redundant_sensors)


100%|██████████| 2057/2057 [00:06<00:00, 295.85it/s]

In [13]:
df_D161075_2016_anemometer_identification['broken?'].value_counts()

None                       15273
Ch1Anem                      117
Ch3Anem                       63
Ch5Anem                       58
Ch1Anem,Ch3Anem,Ch5Anem       40
Ch1Anem,Ch3Anem               28
Ch3Anem,Ch5Anem               25
Ch1Anem,Ch5Anem               10
Name: broken?, dtype: int64

In [22]:
df_D161075_2016_anemometer_identification.name = 'D161075_2016'
generate_dashboard(df_D161075_2016_anemometer_identification, redundance_dict=df_D161075_redundant_sensors)


100%|██████████| 9/9 [00:00<00:00, 405.32it/s]
0it [00:00, ?it/s]
100%|██████████| 5/5 [00:00<00:00, 391.13it/s]
0it [00:00, ?it/s]
100%|██████████| 4/4 [00:00<00:00, 427.19it/s]
0it [00:00, ?it/s]

In [14]:
df_D161075_2017 = pd.read_csv('../Cleaner/data/D161075_2017_data.csv', index_col=0)

df_D161075_2017.sort_values('Timestamp', inplace=True)
df_D161075_2017.reset_index(drop=True, inplace=True)

In [17]:
# Slice de las primeras 7 columnas para no agarrar las columnas con los min y std
df_D161075_2017_fault_detection = fault_detection(df_D161075_2017.iloc[:, :7], sensor='anem', correlation_window=3, ratio_th = 10, correlation_th= .5)


100%|██████████| 15/15 [06:16<00:00, 24.44s/it]

In [19]:
df_D161075_2017_anemometer_identification = anemometer_identification(df_D161075_2017_fault_detection, df_D161075_redundant_sensors)


100%|██████████| 6044/6044 [00:36<00:00, 163.38it/s]

In [21]:
df_D161075_2017_anemometer_identification['broken?'].value_counts()

None                       51724
Ch1Anem                      272
Ch5Anem                      174
Ch3Anem                      124
Ch1Anem,Ch3Anem,Ch5Anem      113
Ch3Anem,Ch5Anem               72
Ch1Anem,Ch3Anem               48
Ch1Anem,Ch5Anem               33
Name: broken?, dtype: int64

In [23]:
df_D161075_2017_anemometer_identification.name = 'D161075_2017'
generate_dashboard(df_D161075_2017_anemometer_identification, redundance_dict=df_D161075_redundant_sensors)


100%|██████████| 16/16 [00:00<00:00, 462.78it/s]
0it [00:00, ?it/s]
100%|██████████| 8/8 [00:00<00:00, 480.92it/s]
0it [00:00, ?it/s]
100%|██████████| 10/10 [00:00<00:00, 515.96it/s]
0it [00:00, ?it/s]

In [24]:
df_D161075_2018 = pd.read_csv('../Cleaner/data/D161075_2018_data.csv', index_col=0)

df_D161075_2018.sort_values('Timestamp', inplace=True)
df_D161075_2018.reset_index(drop=True, inplace=True)

# Slice de las primeras 7 columnas para no agarrar las columnas con los min y std
df_D161075_2018_fault_detection = fault_detection(df_D161075_2018.iloc[:, :7], sensor='anem', correlation_window=3, ratio_th = 10, correlation_th= .5)

df_D161075_2018_anemometer_identification = anemometer_identification(df_D161075_2018_fault_detection, df_D161075_redundant_sensors)

df_D161075_2018_anemometer_identification.name = 'D161075_2018'
generate_dashboard(df_D161075_2018_anemometer_identification, redundance_dict=df_D161075_redundant_sensors)


100%|██████████| 11/11 [00:00<00:00, 536.52it/s]
0it [00:00, ?it/s]
100%|██████████| 9/9 [00:00<00:00, 525.05it/s]
0it [00:00, ?it/s]
100%|██████████| 59/59 [00:00<00:00, 332.66it/s]
0it [00:00, ?it/s]

In [25]:
df_D161075_2018_anemometer_identification['broken?'].value_counts()

None                       34425
Ch5Anem                     7899
Ch1Anem                      128
Ch3Anem                      119
Ch1Anem,Ch5Anem               90
Ch3Anem,Ch5Anem               81
Ch1Anem,Ch3Anem,Ch5Anem       65
Ch1Anem,Ch3Anem               43
Name: broken?, dtype: int64

In [28]:
df_vaquerias = pd.read_csv('../Cleaner/data/vaquerias.csv', index_col=0)

In [30]:
df_vaquerias_redundant_sensors = {
    'Ch1Anem': 'Ch2Anem',
    'Ch2Anem': 'Ch1Anem',
    'Ch3Anem': 'Ch2Anem',
    'Ch4Anem': 'Ch5Anem',
    'Ch5Anem': 'Ch4Anem',
    'Ch6Anem': 'Ch5Anem',
}

In [31]:
df_vaquerias_fault_detection = fault_detection(df_vaquerias, sensor='anem', correlation_window=10, ratio_th = 10, correlation_th= .7)
df_vaquerias_anemometer_identification = anemometer_identification(df_vaquerias_fault_detection, df_vaquerias_redundant_sensors)


100%|██████████| 9654/9654 [00:27<00:00, 355.62it/s]

In [32]:
df_vaquerias_anemometer_identification.name = 'Vaquerias'
generate_dashboard(df_vaquerias_anemometer_identification, redundance_dict=df_vaquerias_redundant_sensors)


100%|██████████| 8/8 [00:00<00:00, 460.48it/s]
0it [00:00, ?it/s]
100%|██████████| 1/1 [00:00<00:00, 372.73it/s]

In [36]:
df_vaquerias_anemometer_identification['broken?'].value_counts()

Ch1Anem                                            6352
None                                               5624
Ch1Anem,Ch4Anem                                     133
Ch4Anem                                              73
Ch1Anem,Ch4Anem,Ch5Anem,Ch6Anem                       7
Ch1Anem,Ch3Anem,Ch4Anem,Ch5Anem,Ch6Anem               5
Ch1Anem,Ch2Anem,Ch3Anem,Ch4Anem,Ch5Anem,Ch6Anem       3
Ch3Anem,Ch4Anem,Ch5Anem,Ch6Anem                       2
Ch1Anem,Ch3Anem                                       1
Ch1Anem,Ch2Anem                                       1
Name: broken?, dtype: int64